In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317145 sha256=db7f24ef58c17c8dd6f744d16e9f2b03ea8698f49e2a09e40afd5680b88363eb
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [ ]:
import pandas as pd
from pyspark.sql.functions import col, explode


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
from pyspark.sql import SparkSession
#spark.stop()
spark = SparkSession \
.builder \
.config("spark.executor.instances", "1")\
.config('spark.driver.memory','1g')\
.config('spark.executor.memory', '1g') \
.getOrCreate()

In [ ]:
df = spark.read.csv('/content/gdrive/MyDrive/musicrecom/melodice/melodice.csv', sep=',', inferSchema=True, header=True, nullValue='NA', nanValue='NA',emptyValue='NA').dropna()
df.show()

+-------+-------+-----+---+----+---+---+---+-----+---+---+-------+-----+--------------------+---------+-----------------+-----------+-----------+--------------------+--------------------+--------------------+--------------------+--------------------+
|game_id|link_id|   EA| EE|  EL| EN| EP| RM|   SA| VD| VU|current|valid|          game_title|game_year|game_playing_time|game_rating|game_voters|           game_desc|           game_tags|          link_title|            link_url|       link_duration|
+-------+-------+-----+---+----+---+---+---+-----+---+---+-------+-----+--------------------+---------+-----------------+-----------+-----------+--------------------+--------------------+--------------------+--------------------+--------------------+
|  15504|    445|  3.0|0.0| 0.0|0.0|0.0|0.0|  3.0|0.0|0.0|      0|    0|         Goblin King|     1996|              180|        6.8|         20|"In ""Goblin King...| with their small...| come down from t...|               towns|             castl

In [ ]:
df = spark.read.csv('/content/gdrive/MyDrive/musicrecom/melodice/melodice_all_events.csv', sep=',', inferSchema=True, header=True, nullValue='NA', nanValue='NA',emptyValue='NA').dropna()


In [ ]:
df.show()

+-------+-------+---+--------+--------------------+----------+
|game_id|link_id|idx|cookieid|          event_date|event_type|
+-------+-------+---+--------+--------------------+----------+
|    242|   3260| -1|     0-0|2016-05-19 18:12:...|        EA|
|    242|   3261| -1|     0-0|2016-05-19 18:18:...|        EA|
|    242|   3262| -1|     0-0|2016-05-19 18:19:...|        EA|
|    242|   3263| -1|     0-0|2016-05-19 18:19:...|        EA|
|    242|   3264| -1|     0-0|2016-05-19 18:21:...|        EA|
|    242|   3265| -1|     0-0|2016-05-19 18:50:...|        EA|
|    242|   3267| -1|     0-0|2016-05-19 18:56:...|        EA|
|    242|   3260| -1|     0-0|2016-05-19 18:59:...|        EA|
|    242|   3261| -1|     0-0|2016-05-19 19:04:...|        EA|
|    242|   3262| -1|     0-0|2016-05-19 19:05:...|        EA|
|    242|   3263| -1|     0-0|2016-05-19 19:06:...|        EA|
|    242|   3264| -1|     0-0|2016-05-19 19:08:...|        EA|
|    242|   3265| -1|     0-0|2016-05-19 19:13:...|    

In [ ]:
from pyspark.sql.functions import countDistinct


In [ ]:
listened = df.filter(df.event_type == "EA")
listened.show()

+-------+-------+---+--------+--------------------+----------+
|game_id|link_id|idx|cookieid|          event_date|event_type|
+-------+-------+---+--------+--------------------+----------+
|    242|   3260| -1|     0-0|2016-05-19 18:12:...|        EA|
|    242|   3261| -1|     0-0|2016-05-19 18:18:...|        EA|
|    242|   3262| -1|     0-0|2016-05-19 18:19:...|        EA|
|    242|   3263| -1|     0-0|2016-05-19 18:19:...|        EA|
|    242|   3264| -1|     0-0|2016-05-19 18:21:...|        EA|
|    242|   3265| -1|     0-0|2016-05-19 18:50:...|        EA|
|    242|   3267| -1|     0-0|2016-05-19 18:56:...|        EA|
|    242|   3260| -1|     0-0|2016-05-19 18:59:...|        EA|
|    242|   3261| -1|     0-0|2016-05-19 19:04:...|        EA|
|    242|   3262| -1|     0-0|2016-05-19 19:05:...|        EA|
|    242|   3263| -1|     0-0|2016-05-19 19:06:...|        EA|
|    242|   3264| -1|     0-0|2016-05-19 19:08:...|        EA|
|    242|   3265| -1|     0-0|2016-05-19 19:13:...|    

In [18]:
listened.count()

30504155

In [ ]:
upvoted = df.filter(df.event_type == "VU")
upvoted.count()

26716

In [ ]:
downvoted = df.filter(df.event_type == "VD")
downvoted.count()

17190

In [ ]:
skipped = df.filter((df.event_type == "EP") | (df.event_type == "EN") | (df.event_type == "EL"))
skipped.count()

2483347

In [ ]:
from pyspark.sql.functions import countDistinct, count


In [ ]:
user_song = listened.groupBy("link_id", "cookieid").count()

In [ ]:
user_song = user_song.withColumnRenamed("count", "listening_count")

In [ ]:
user_listenings = user_song.groupBy("cookieid").sum("listening_count").withColumnRenamed("sum(listening_count)", "sum_listenings")

In [19]:
user_listenings.count()

1624896

In [20]:
# upvoted.write.option("header",True).csv("/content/gdrive/MyDrive/musicrecom/melodice/upvoted.csv")


In [ ]:
# downvoted.write.option("header",True).csv("/content/gdrive/MyDrive/musicrecom/melodice/downvoted.csv")


In [ ]:
# skipped.write.option("header",True).csv("/content/gdrive/MyDrive/musicrecom/melodice/skipped.csv")


In [ ]:
# user_song.write.option("header",True).csv("/content/gdrive/MyDrive/musicrecom/melodice/user_song_2.csv")

In [ ]:
# user_listenings.write.option("header",True).csv("/content/gdrive/MyDrive/musicrecom/melodice/user_listenings.csv")